Notebook to split the stump\
Press "Run All", visualize the output and both files(left and right stump) will be downloaded on your desktop
 

In [ ]:
import numpy as np
from stl import mesh
import open3d as o3d
import os

Aligns the stump to the y-axis, can be aligned to any axis needed\
Already in the "wrapping" code for the z axis

In [ ]:
def align_to_y_axis(stl_mesh):

    vertices = np.asarray(stl_mesh.vectors).reshape(-1, 3)
    center_of_mass = np.mean(vertices, axis=0)
    centered_vertices = vertices - center_of_mass  
    cov = np.cov(centered_vertices.T)  
    eigenvalues, eigenvectors = np.linalg.eig(cov)  

    # Extract the principal axis
    principal_axis = eigenvectors[:, np.argmax(eigenvalues)]  # Principal axis

    y_axis = np.array([0, 1, 0])  
    v = np.cross(principal_axis, y_axis)  
    s = np.linalg.norm(v) 
    if s < 1e-6:
        return stl_mesh  

    c = np.dot(principal_axis, y_axis)  
    v_skew = np.array([
        [0, -v[2], v[1]],
        [v[2], 0, -v[0]],
        [-v[1], v[0], 0]
    ])  
    rotation_matrix = np.eye(3) + v_skew + np.dot(v_skew, v_skew) * ((1 - c) / (s ** 2))

    for i, triangle in enumerate(stl_mesh.vectors):
        stl_mesh.vectors[i] = np.dot(triangle - center_of_mass, rotation_matrix.T) + center_of_mass

    translation = [-center_of_mass[0], 0, -center_of_mass[2]]
    stl_mesh.translate(translation)
    return stl_mesh

Splits the stumps along the y-axis

In [ ]:
def split_stl_parallel_to_y_axis(input_file, output_file_1, output_file_2):

    stl_mesh = mesh.Mesh.from_file(input_file)

    # Align the mesh to the Y-axis
    aligned_mesh = align_to_y_axis(stl_mesh)

    
    triangles_left = []
    triangles_right = []
    for triangle in aligned_mesh.vectors:
        centroid = np.mean(triangle, axis=0)
        if centroid[0] >= 0: 
            triangles_right.append(triangle)
        else:  
            triangles_left.append(triangle)


    left_mesh = mesh.Mesh(np.zeros(len(triangles_left), dtype=mesh.Mesh.dtype))
    right_mesh = mesh.Mesh(np.zeros(len(triangles_right), dtype=mesh.Mesh.dtype))

    for i, triangle in enumerate(triangles_left):
        left_mesh.vectors[i] = triangle
    for i, triangle in enumerate(triangles_right):
        right_mesh.vectors[i] = triangle

    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")  # Get desktop path dynamically
    left_mesh.save(os.path.join(desktop_path, output_file_1))
    right_mesh.save(os.path.join(desktop_path, output_file_2))

    print(f"Saved left part to {os.path.join(desktop_path, output_file_1)}")
    print(f"Saved right part to {os.path.join(desktop_path, output_file_2)}")

    # Visualize both parts using Open3D
    visualize_stl_parts_open3d(os.path.join(desktop_path, output_file_1), os.path.join(desktop_path, output_file_2))

Saves the Left and Right part of the stump on your desktop (on MacOS, maybe not on windows idk ...)

In [ ]:
def visualize_stl_parts_open3d(file1, file2):
    mesh1 = o3d.io.read_triangle_mesh(file1)
    mesh2 = o3d.io.read_triangle_mesh(file2)

    mesh1.paint_uniform_color([1, 0, 0])  # Red
    mesh2.paint_uniform_color([0, 0, 1])  # Blue

    o3d.visualization.draw_geometries([mesh1, mesh2], window_name="Split STL Visualization")

Defines the files & calls the functions

In [ ]:
input_stl = "stump.stl"  # Input file name
output_stl_left = "stump_left.stl"  # Left part output file name
output_stl_right = "stump_right.stl"  # Right part output file name

split_stl_parallel_to_y_axis(input_stl, output_stl_left, output_stl_right)